In [1]:
import numpy as np
import os

In [2]:
img_width=150
img_height=150

In [3]:
from tensorflow.keras import backend as K

if K.image_data_format() == 'channels_first': 
    input_shape = (3, img_width, img_height) 
else: 
    input_shape = (img_width, img_height, 3) 
print(input_shape)

(150, 150, 3)


In [4]:
path="C:\\Users\\rexdi\\Desktop\\flowers\\"

train_dir=path+'\\Train\\'
valid_dir=path+'\\Valid'

epochs=10
batch_size=100

In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
#Image agumentation
train_datagen = ImageDataGenerator( 
                rescale = 1. / 255, 
                 shear_range = 0.2, 
                  zoom_range = 0.2, 
            horizontal_flip = True) 
  
test_datagen = ImageDataGenerator(rescale = 1. / 255) 

In [6]:
train_generator = train_datagen.flow_from_directory(train_dir, 
                              target_size =(img_width, img_height), 
                     batch_size = batch_size, class_mode ='categorical') 
  
validation_generator = test_datagen.flow_from_directory( 
                                    valid_dir, 
                   target_size =(img_width, img_height), 
          batch_size = batch_size, class_mode ='categorical') 

Found 4323 images belonging to 5 classes.
Found 1468 images belonging to 5 classes.


In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Conv2D,Flatten,MaxPooling2D,Dropout,Activation
from tensorflow.keras.optimizers import Adam

In [8]:
model=Sequential()

model.add(Conv2D(32,(3,3),input_shape=(150,150,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(128,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(256,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(3,3)))

model.add(Flatten())

model.add(Dense(units=128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(units=5, activation='softmax'))
opt = Adam(lr=1e-3, decay=1e-6)
model.compile(metrics=['accuracy'],optimizer='adam',loss='categorical_crossentropy')

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 72, 72, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 34, 34, 128)       73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 17, 17, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 15, 15, 256)       2

In [9]:
train_generator.classes.shape,validation_generator.classes.shape

((4323,), (1468,))

In [ ]:
op = model.fit_generator(train_generator,epochs=48,validation_data=validation_generator)

Epoch 1/48
44/44 [==============================] - 28s 638ms/step - loss: 1.4171 - accuracy: 0.3736 - val_loss: 1.2298 - val_accuracy: 0.4557
Epoch 2/48
44/44 [==============================] - 24s 555ms/step - loss: 1.1997 - accuracy: 0.4902 - val_loss: 1.0735 - val_accuracy: 0.5497
Epoch 3/48
44/44 [==============================] - 25s 557ms/step - loss: 1.0637 - accuracy: 0.5702 - val_loss: 0.9477 - val_accuracy: 0.6172
Epoch 4/48
44/44 [==============================] - 26s 590ms/step - loss: 0.9567 - accuracy: 0.6359 - val_loss: 0.9005 - val_accuracy: 0.6437
Epoch 5/48
44/44 [==============================] - 25s 566ms/step - loss: 0.9247 - accuracy: 0.6563 - val_loss: 0.8824 - val_accuracy: 0.6621
Epoch 6/48
44/44 [==============================] - 25s 560ms/step - loss: 0.9246 - accuracy: 0.6526 - val_loss: 0.8228 - val_accuracy: 0.6860
Epoch 7/48
44/44 [==============================] - 24s 552ms/step - loss: 0.8234 - accuracy: 0.6840 - val_loss: 0.7876 - val_accuracy: 0.6982

In [ ]:
model.save('op.h5')

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns

%matplotlib inline

plt.plot(op.history['loss'])
plt.plot(op.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epochs')
plt.legend(['train', 'test'])
plt.show()

In [ ]:
plt.plot(op.history['accuracy'])
plt.plot(op.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.legend(['train', 'test'])
plt.show()

In [ ]:
from tensorflow.keras.preprocessing.image import load_img,img_to_array
#Data Load
flower_test=load_img(path+'\\b_rose.jpg',target_size=(150,150))
#print(flower_test.format)

flower_img_array=img_to_array(flower_test)
print('Conversion to array',flower_img_array.shape)

flower = np.expand_dims(flower_img_array, axis=0)         #Pre processing
print('Dimension expansion',flower.shape)

pred=model.predict(flower)
op_flower=np.argmax(pred,axis=1)

if op_flower==0:
    print('Daisy')
elif op_flower==1:
    print('Dandelion')
elif op_flower==2:
    print('Rose')
elif op_flower==3:
    print('Sunflower')
elif op_flower==4:
    print('Tulip')